### 7.基于位置的数组修改器

In [1]:
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
a = {
    'content': '...',
    'comments': [
        {
            'comment': 'good post',
            'author': 'John',
            'votes': 0
        },
        {
            'comment': 'i thought it was too short',
            'author': 'Claire',
            'votes': 3
        },
        {
            'comment': 'free watches',
            'author': 'Alice',
            'votes': -1
        }
    ]
}

In [3]:
client = MongoClient()

In [4]:
db = client.test

In [5]:
t = db.t

In [6]:
t.drop()

In [7]:
t.insert_one(a)

In [8]:
from pprint import pprint
pprint(t.find_one())

{'_id': ObjectId('59ae3145498ab12382485026'),
 'comments': [{'author': 'John', 'comment': 'good post', 'votes': 0},
              {'author': 'Claire',
               'comment': 'i thought it was too short',
               'votes': 3},
              {'author': 'Alice', 'comment': 'free watches', 'votes': -1}],
 'content': '...'}


In [9]:
t.update_one({'comments.author': 'John'},
             {'$set': {'comments.$.author': 'Jim'}})

In [10]:
pprint(t.find_one())

{'_id': ObjectId('59ae3145498ab12382485026'),
 'comments': [{'author': 'Jim', 'comment': 'good post', 'votes': 0},
              {'author': 'Claire',
               'comment': 'i thought it was too short',
               'votes': 3},
              {'author': 'Alice', 'comment': 'free watches', 'votes': -1}],
 'content': '...'}


### 8.修改器速度

In [11]:
t.drop()

In [12]:
t.insert_one({'x': 'a'})

In [13]:
t.insert_one({'x': 'b'})

In [14]:
t.insert_one({'x': 'c'})

In [15]:
[item for item in t.find()]

[{'_id': ObjectId('59ae3145498ab12382485027'), 'x': 'a'},
 {'_id': ObjectId('59ae3145498ab12382485028'), 'x': 'b'},
 {'_id': ObjectId('59ae3145498ab12382485029'), 'x': 'c'}]

In [16]:
t.update_one({'x': 'b'},
             {'$set': {'x': 'bbb'}})

In [17]:
[item for item in t.find()]

[{'_id': ObjectId('59ae3145498ab12382485027'), 'x': 'a'},
 {'_id': ObjectId('59ae3145498ab12382485028'), 'x': 'bbb'},
 {'_id': ObjectId('59ae3145498ab12382485029'), 'x': 'c'}]

In [18]:
t.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 't.stats')

In [19]:
t.paddingFactor

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 't.paddingFactor')

In [20]:
t.drop()

In [21]:
t.find_one()

In [22]:
t.insert_one({'x': 1})

In [23]:
t.find_one()

{'_id': ObjectId('59ae3145498ab1238248502a'), 'x': 1}

In [25]:
from time import time
begin = time()

for i in range(10000):
    t.update_one({'_id': ObjectId('59ae2cdb498ab12130844b68')}, {'$inc': {'x': 1}})

end = time()
print(end - begin)

3.9851672649383545


In [26]:
t.drop()
t.insert_one({'x': 1})
t.find_one()

{'_id': ObjectId('59ae3189498ab1238248502b'), 'x': 1}

In [27]:
from time import time
begin = time()

for i in range(10000):
    t.update_one({'_id': ObjectId('59ae2cdb498ab12130844b68')}, {'$push': {'x': 1}})

end = time()
print(end - begin)

4.375843048095703


## 3.3.3 upsert

In [74]:
t.drop()

In [75]:
blog = t.find_one({'url': 1})

In [76]:
if blog:
    t.update_one({},
                 {'$inc': {'url': 1}})
else:
    t.insert_one({'url': 1})

In [77]:
t.find_one()

{'_id': ObjectId('59ae35b5498ab1238248502f'), 'url': 1}

In [94]:
t.drop()
t.update_one({'url': 0},
             {'$inc': {'url': 1}},
             True)
t.find_one()

{'_id': ObjectId('59ae36f61852c5217bbea4e0'), 'url': 1}

In [90]:
t.drop()
t.update_one({'url': 0},
             {'$inc': {'url': 1}})
t.find_one()

In [95]:
t.drop()

In [96]:
t.update_one({'rep': 25},
             {'$inc': {'rep': 3}},
             True)

In [97]:
t.find_one()

{'_id': ObjectId('59ae37381852c5217bbea4f1'), 'rep': 28}

In [145]:
t.drop()

In [146]:
t.update_one({},
             {'$setOnInsert': {'createdAt': time()}},
             True)
t.find_one()

{'_id': ObjectId('59ae40fd1852c5217bbea718'), 'createdAt': 1504592125.666772}

In [147]:
t.update_one({},
             {'$setOnInsert': {'createdAt': time()}},
             True)
t.find_one()

{'_id': ObjectId('59ae40fd1852c5217bbea718'), 'createdAt': 1504592125.666772}

### save shell 帮助程序

In [154]:
t.drop()

In [155]:
x = 42

In [156]:
t.save(x)

/Users/wonderful/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  if __name__ == '__main__':


TypeError: to_save must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

## 3.3.4 更新多个文档

In [199]:
t.drop()

In [200]:
a = {'bitrhday': '10/13/1978'}
b = {'bitrhday': '10/13/1978'}
c = {'bitrhday': '10/13/1978'}
d = {'bitrhday': '10/12/1978'}

In [201]:
t.insert_many([a, b, c, d])

In [202]:
from pprint import pprint

In [203]:
[_ for _ in t.find()]

[{'_id': ObjectId('59ae46b3498ab1238248503f'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46b3498ab12382485040'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46b3498ab12382485041'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46b3498ab12382485042'), 'bitrhday': '10/12/1978'}]

In [195]:
t.update_one({'bitrhday': '10/13/1978'}, 
             {'$set': {'gift': 'Happy Birthday!'}},
             True,
             False)

In [196]:
[_ for _ in t.find()]

[{'_id': ObjectId('59ae46a6498ab1238248503b'),
  'bitrhday': '10/13/1978',
  'gift': 'Happy Birthday!'},
 {'_id': ObjectId('59ae46a6498ab1238248503c'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46a6498ab1238248503d'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46a6498ab1238248503e'), 'bitrhday': '10/12/1978'}]

In [197]:
t.update_one({'bitrhday': '10/13/1978'}, 
             {'$set': {'gift': 'Cake'}},
             False,
             True)

In [198]:
[_ for _ in t.find()]

[{'_id': ObjectId('59ae46a6498ab1238248503b'),
  'bitrhday': '10/13/1978',
  'gift': 'Cake'},
 {'_id': ObjectId('59ae46a6498ab1238248503c'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46a6498ab1238248503d'), 'bitrhday': '10/13/1978'},
 {'_id': ObjectId('59ae46a6498ab1238248503e'), 'bitrhday': '10/12/1978'}]

In [187]:
help(t.update_one)

Help on method update_one in module pymongo.collection:

update_one(filter, update, upsert=False, bypass_document_validation=False, collation=None) method of pymongo.collection.Collection instance
    Update a single document matching the filter.
    
      >>> for doc in db.test.find():
      ...     print(doc)
      ...
      {u'x': 1, u'_id': 0}
      {u'x': 1, u'_id': 1}
      {u'x': 1, u'_id': 2}
      >>> result = db.test.update_one({'x': 1}, {'$inc': {'x': 3}})
      >>> result.matched_count
      1
      >>> result.modified_count
      1
      >>> for doc in db.test.find():
      ...     print(doc)
      ...
      {u'x': 4, u'_id': 0}
      {u'x': 1, u'_id': 1}
      {u'x': 1, u'_id': 2}
    
    :Parameters:
      - `filter`: A query that matches the document to update.
      - `update`: The modifications to apply.
      - `upsert` (optional): If ``True``, perform an insert if no documents
        match the filter.
      - `bypass_document_validation`: (optional) If ``True``, 